In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
import pkg_resources
from spacy.lang.en.stop_words import STOP_WORDS

import nltk
from nltk.stem import WordNetLemmatizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
reviews = pd.read_csv("/content/drive/MyDrive/RMP/scraped_comments_with_professor.csv")
# reviews[['comment_id', 'firstName', 'lastName', 'prof_class', 'comment', 'clarityRating', 'helpfulRating']].to_csv("/content/drive/MyDrive/RMP/scraped_comments_sample.csv")

In [ ]:
reviews.head()

,professor_id,comment_id,firstName,lastName,prof_class,comment,ratingTags,date,attendanceMandatory,grade,clarityRating,difficultyRating,helpfulRating,textbookUse,thumbsDownTotal,thumbsUpTotal,wouldTakeAgain
0,VGVhY2hlci0xMjQzMzQ3,UmF0aW5nLTIyMTY1MTI1,Kurt,Douglass,ENGLISH125,"Good teacher, good lectures. Obviously cares a...",NaN,2013-10-08 21:49:34 +0000 UTC,N,A,5,2,5,4.0,0,0,NaN
1,VGVhY2hlci0xMjQzMzQ3,UmF0aW5nLTIyMTY1MTEx,Kurt,Douglass,ENG125,"Good teacher, very lenient with grading and at...",NaN,2013-10-08 21:42:25 +0000 UTC,Y,A,5,2,5,4.0,0,0,NaN
2,VGVhY2hlci0xMjQzMzQ3,UmF0aW5nLTE1ODQ1MTIy,Kurt,Douglass,ENG11,Very difficult class. His grading is hard to ...,NaN,2009-05-15 13:59:15 +0000 UTC,NaN,NaN,1,5,2,5.0,0,0,NaN
3,VGVhY2hlci0xMDExMDU2,UmF0aW5nLTIxMjExOTYy,Paula,Zobisch,MGT5000,Excellent mentor. Created valuable foundations...,NaN,2013-01-05 10:30:39 +0000 UTC,NaN,NaN,5,1,5,5.0,0,0,NaN
4,VGVhY2hlci0xMDExMDU2,UmF0aW5nLTE3MjQxOTI2,Paula,Zobisch,MKT401,Awesome...Had her for two classes at baker col...,NaN,2010-05-20 14:39:01 +0000 UTC,NaN,NaN,5,3,5,3.0,1,0,NaN


In [ ]:
print("Shape before dropping:", reviews.shape)
reviews.drop_duplicates(subset="comment_id", keep="first", inplace=True)

# drop rows containing only "No Comments" (default value assigned by RMP to a review that didn't enter a comment)
reviews = reviews[reviews["comment"] != "No Comments"]
# drop rows containing NaN comment
reviews.dropna(subset=["comment"], inplace=True)
# drop rows where clarityRating == 3
reviews = reviews[reviews['clarityRating'] != 3]

print("Shape after dropping:", reviews.shape)
reviews.reset_index(drop=True, inplace=True)

Shape before dropping: (475073, 17)
Shape after dropping: (415399, 17)


In [ ]:
prof_counts = reviews['professor_id'].value_counts()
prof_counts

VGVhY2hlci0xMDAxMDQw    506
VGVhY2hlci0xNTk2ODg1    371
VGVhY2hlci0xMjgyNTU0    326
VGVhY2hlci0xMjM4MjY5    326
VGVhY2hlci0xNDE3NjAy    285
                       ... 
VGVhY2hlci0xMzU4MDk2      1
VGVhY2hlci0xNTY3MzA5      1
VGVhY2hlci0xNDYzNTg4      1
VGVhY2hlci0xMzUzMDYz      1
VGVhY2hlci0xMzUzNDI5      1
Name: professor_id, Length: 54014, dtype: int64

In [ ]:
prof_counts.describe()

count    54014.000000
mean         7.690580
std         10.709292
min          1.000000
25%          2.000000
50%          5.000000
75%          9.000000
max        506.000000
Name: professor_id, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(prof_counts.index, prof_counts.values, test_size=0.2, random_state=1)

In [ ]:
print(prof_counts[X_train].sum(), prof_counts[X_test].sum(), prof_counts.sum())

331104 84295 415399


In [ ]:
x_train_set = set(X_train)
x_test_set = set(X_test)
print(x_train_set.intersection(x_test_set)) # Set is empty, meaning that 

set()


In [ ]:
reviews['sentiment'] = reviews['clarityRating'] >= 3

In [ ]:
reviews_train = reviews[reviews['professor_id'].isin(X_train)]
reviews_test = reviews[reviews['professor_id'].isin(X_test)]
print(reviews_train.shape)
print(reviews_test.shape)

(331104, 18)
(84295, 18)


In [ ]:
print(reviews_train.sentiment.sum()/reviews_train.shape[0])
print(reviews_test.sentiment.sum()/reviews_test.shape[0])

0.746052599787378
0.7468889020701109
